In [4]:
import pandas as pd
import re
import numpy as np
from utils.configurar_logger import configurar_logger
from utils.manejador_bucket import subir_archivo_a_space
from utils.manejador_bucket import listar_archivos_en_folder
from utils.manejador_bucket import leer_archivo_desde_space
from functools import reduce
from datetime import datetime
import missingno as msno

In [5]:
logger = configurar_logger("../logs/consumer.log")

def extraer_datos_de_folder(ruta_folder):
    archivos = listar_archivos_en_folder(ruta_folder)
    if len(archivos)<=1:
        logger.info(f"No hay archivos en la carpeta '{ruta_folder}'")
        return None
    data_completa = []
    for archivo in archivos[1:]:
        if not archivo.endswith(".json"):
            continue
        data = leer_archivo_desde_space(archivo)
        if isinstance(data, list):
            data_completa.extend(data)
        elif isinstance(data, dict):
            data_completa.append(data)
        else:
            logger.warning(f"Formato de datos no reconocido en el archivo {archivo}")

    if not data_completa:
        logger.info(f"No se encontraron datos válidos en la carpeta '{ruta_folder}'")
        return None
    return data_completa


def convertir_a_dataframe(data):
    variables_comunes = reduce(
        lambda x, y: x & set(y.keys()),
        data,
        set(data[0].keys())
    )

    data_comun = reduce(
        lambda x, y: x+[{k: y[k] for k in variables_comunes if k in y}],
        data,
        []
    )
    return pd.DataFrame(data_comun)



def coordenadas_format(df: pd.DataFrame,lat_lon: list) -> np.array:
    '''
    :param df:  
    :param lat_lon: 
    :return: 
    '''
    coordenadas = df[lat_lon].values
    coordenadas_f = []
    for coord in coordenadas:
        if coord[0] is None or coord[1] is None:
            coordenadas_f.append(np.array([None, None]))
            continue
        lat_f = float(coord[0].replace(",","."))
        lon_f = float(coord[1].replace(",","."))
        coordenadas_f.append(np.array([lat_f, lon_f]))
    return np.array(coordenadas_f)

Cargamos los archivos que se encuentran rawdata

In [6]:
ruta_folder = f"precios-depas-lima/rawdata/"
data_json = extraer_datos_de_folder(ruta_folder)
data = convertir_a_dataframe(data_json)
data

2025-02-17 20:29:31,220 - INFO - Hay 235  archivos en el folder


,fecha_publicacion,banios/2,latitud,antiguedad,dormitorios,area_cubierta,precio_dolares,longitud,banos,descripcion,estacionamiento,area_total,precio_soles,distrito
0,Publicado hoy,None,-12.139912400000000,5 años,3 dorm.,77 m² cub.,USD 800,-77.018605600000000,2 baños,Se Alquila Lindo Departamento en Barranco\n¡De...,1 estac.,77 m² tot.,"S/ 2,960",barranco
1,Publicado hoy,None,-12.139912400000000,5 años,3 dorm.,77 m² cub.,USD 800,-77.018605600000000,2 baños,Se Alquila Lindo Departamento en Barranco\n¡De...,1 estac.,77 m² tot.,"S/ 2,960",barranco
2,Publicado desde ayer,None,-12.139271799999999,A estrenar,2 dorm.,96 m² cub.,USD 600,-77.021153799999993,2 baños,Se Alquila Departamento Duplex en Corazón de B...,1 estac.,96 m² tot.,"S/ 2,250",barranco
3,Publicado desde ayer,1 medio baño,-12.139554600000000,8 años,1 dorm.,40 m² cub.,USD 535,-77.023879399999998,1 baño,Se Alquila Lindo Departamento en Barranco\nSE ...,1 estac.,40 m² tot.,"S/ 2,000",barranco
4,Publicado desde ayer,None,-12.138541000000000,10 años,3 dorm.,80 m² cub.,USD 800,-77.017627400000000,2 baños,Departamento de 3 Habitaciones Sin Amoblar en ...,None,80 m² tot.,"S/ 2,974",barranco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,Publicado hace 19 días,None,-12.102243900000000,A estrenar,2 dorm.,52 m² cub.,None,-77.020318000000000,2 baños,Hermoso Departamento de Estreno en El Corazon ...,None,52 m² tot.,"S/ 2,200",surquillo
5527,Publicado hace 94 días,None,-12.120323300000000,A estrenar,3 dorm.,67 m² cub.,USD 784,-77.006593600000002,2 baños,Departamento Estreno en Alquiler con Accesos I...,None,67 m² tot.,"S/ 2,600",surquillo
5528,Publicado hace 13 días,None,-12.118281900000000,2 años,3 dorm.,74 m² cub.,USD 900,-77.015435200000000,2 baños,"Ocasion, Alquilo Departamento 3dorm, Amoblado,...",1 estac.,74 m² tot.,"S/ 3,400",surquillo
5529,Publicado hace 125 días,None,-12.118683000000000,2 años,3 dorm.,None,USD 740,-77.007747699999995,2 baños,Alquilo Departamento Amoblado en Tomas Marzano...,1 estac.,75 m² tot.,"S/ 2,800",surquillo


Veamos cuales son las unidades de medida (formato) de las variables.

In [7]:
columnas = data.columns.tolist()
columnas.remove("descripcion")
columnas.remove("distrito")
unidades_col = {}
for col in columnas:
    valores = data[col].values
    unidades = []
    for valor in valores:
        if valor is None:
            continue
        unidad = ""
        caracteres = re.findall(r"\D+", valor)
        for caracter in caracteres:
            unidad += caracter
        unidades.append(unidad)
    unidades_col[col] = set(unidades)

Ahora veamos algunos ejemplos de los tipos de valores (formatos) existentes en cada columna segun los formatos obtenidos anteriormente.

In [26]:
columnas = data.columns.tolist()
columnas.remove("descripcion")
columnas.remove("distrito")
unidades_col = {}
for col in columnas:
    valores = data[col].values
    unidades = []
    for valor in valores:
        if valor is None:
            continue
        unidad = ""
        caracteres = re.findall(r"\D+", valor)
        for caracter in caracteres:
            unidad += caracter
        unidades.append(unidad)
    unidades_col[col] = set(unidades)

def tipos_formato(data):
    ejemplos = {}
    
    # Iteramos sobre cada columna en la que se extrajeron unidades.
    for col in unidades_col:
        ejemplos[col] = {}  # Inicializamos un diccionario para almacenar ejemplos por unidad.
        # Para cada unidad (formato) extraída de la columna:
        for unidad in unidades_col[col]:
            ejemplos_col = []
            # Recorremos los valores de la columna, omitiendo los nulos
            for valor in data[col].dropna().astype(str):
                # Extraemos la parte no numérica (la unidad) del valor.
                unidad_extraida = "".join(re.findall(r"\D+", valor))
                # Si coincide con la unidad buscada, lo añadimos a los ejemplos.
                if unidad_extraida == unidad:
                    ejemplos_col.append(valor)
                # Limitar a 3 ejemplos por unidad
                #if len(ejemplos_col) >= 3:
    #                break
            ejemplos[col][unidad] = pd.Series(ejemplos_col).unique()[:3]
    
    # Ejemplo de impresión de resultados:
    for col, unidades in ejemplos.items():
        print(f"\n------------ Columna: {col} --------------- ")
        for unidad, vals in unidades.items():
            print(f"  Unidad: '{unidad}' -> Ejemplos: {vals}")

In [27]:
tipos_formato(data)


------------ Columna: fecha_publicacion --------------- 
  Unidad: 'Hace  días' -> Ejemplos: ['Hace 2 días' 'Hace 3 días' 'Hace 4 días']
  Unidad: 'Publicado desde ayer' -> Ejemplos: ['Publicado desde ayer']
  Unidad: 'Publicado hace más de  año' -> Ejemplos: ['Publicado hace más de 1 año']
  Unidad: 'Publicado hace  días' -> Ejemplos: ['Publicado hace 2 días' 'Publicado hace 3 días' 'Publicado hace 4 días']
  Unidad: 'Hace + días' -> Ejemplos: ['Hace +30 días']
  Unidad: 'Hace  día' -> Ejemplos: ['Hace 1 día']
  Unidad: 'Publicado hoy' -> Ejemplos: ['Publicado hoy']
  Unidad: 'Hace  h  minutos' -> Ejemplos: ['Hace 20 h 15 minutos' 'Hace 20 h 37 minutos' 'Hace 17 h 15 minutos']

------------ Columna: banios/2 --------------- 
  Unidad: ' medios baños' -> Ejemplos: ['3 medios baños' '4 medios baños' '2 medios baños']
  Unidad: ' medio baño' -> Ejemplos: ['1 medio baño']

------------ Columna: latitud --------------- 
  Unidad: '-,' -> Ejemplos: ['-12,0493603' '-12,07908' '-12,0787147']

Apartir de la celda anterior tenemos que tener las siguientes consideraciones:

1. fecha_publicacion: Existen diferentes formatos para esta variable, pero la mayoria se encuentr en dias, por lo que podemos pasar todos los otros formatos a "hace \d+ dias".
2. latitud y longitud : Existe una coma en lugar de punto punto en las latitudes y longitudes, por lo que no se podra interpretar a la hora de hacer la conversion a float.
3. dormitorios, banos, antiguedad, area_total : Existen valores colados de otras columnas, por lo que vamos a restringir los valores de estos al definir la expresion regular.
4. precio_soles : Podemos ver que ademas de soles existe dolares como moneda, por lo que tendremos que almacenar la moneda en otra columna para que una vez formateados podamos converitr todo a soles. Ademas podemos ver ciertos precios exhorbitantes, que posiblemente esten relacionados a la compra de un departamento en lugar de un alquiler. Por ultimo podemos ver que existen ciertos valores que usan el punto para separar los millares, lo cual hara que se considere como decimales.

Apartir de los ejemplos anteriores vamos a definir expresiones regulares para extraer el valor numerico y poder formatear las variables. Sin embargo, antes vamos a realizar un pre-formateo para 

In [75]:
def pre_formateo(df: pd.DataFrame) -> pd.DataFrame:
    df_copia = df.copy()
    
    # fecha_publicacion : convertiremos todos los valores a "hace \d+ días"
    df_copia["fecha_publicacion"] = df_copia["fecha_publicacion"].replace(
        {
            "Publicado desde ayer": "hace 1 día",
            "Publicado hace más de 1 año": "hace 365 días",
            "Publicado hoy": "hace 0 días"
        }
    )
    df_copia["fecha_publicacion"] = np.where(df_copia["fecha_publicacion"].str.contains("minutos"),"hace 0 días",df_copia["fecha_publicacion"])

    # antiguedad
    df_copia['antiguedad'] = np.where(df_copia['antiguedad']=='A estrenar','0 años',df_copia['antiguedad'])
    
    # precio (soles o dolares)
    df_copia['moneda'] = [x[:3] if x else None for x in df_copia['precio_soles'].values]
    df_copia['precio_soles'] = (df_copia['precio_soles'].str.replace(",","")
                                .str.replace(".",""))
    df_copia['precio_dolares'] = (df_copia['precio_dolares'].str.replace(",","")
                                  .str.replace(".",""))
    df_copia['precio_dolares'] = np.where(df_copia['precio_dolares']=='',None, df_copia['precio_dolares'])
    
    # latitud y longitud
    df_copia['latitud'] = df_copia['latitud'].str.replace(",", ".")
    df_copia['longitud'] = df_copia['longitud'].str.replace(",", ".")
    
    # restringir
    df_copia['antiguedad'] = np.where(df_copia['antiguedad'].str.lower().str.contains("año"),
                                      df_copia['antiguedad'], None)
    df_copia['dormitorios'] = np.where(df_copia['dormitorios'].str.lower().str.contains("dorm."),
                                      df_copia['dormitorios'], None)
    df_copia['banos'] = np.where(df_copia['banos'].str.lower().str.contains("baño"),
                                       df_copia['banos'], None)
    df_copia['area_total'] = np.where(df_copia['area_total'].str.lower().str.contains("m²"),
                                      df_copia['area_total'], None)
    
    return df_copia

In [76]:
data_preformato = pre_formateo(data)
tipos_formato(data_preformato)


------------ Columna: fecha_publicacion --------------- 
  Unidad: 'Hace  días' -> Ejemplos: ['Hace 2 días' 'Hace 3 días' 'Hace 4 días']
  Unidad: 'Publicado desde ayer' -> Ejemplos: []
  Unidad: 'Publicado hace más de  año' -> Ejemplos: []
  Unidad: 'Publicado hace  días' -> Ejemplos: ['Publicado hace 2 días' 'Publicado hace 3 días' 'Publicado hace 4 días']
  Unidad: 'Hace + días' -> Ejemplos: ['Hace +30 días']
  Unidad: 'Hace  día' -> Ejemplos: ['Hace 1 día']
  Unidad: 'Publicado hoy' -> Ejemplos: []
  Unidad: 'Hace  h  minutos' -> Ejemplos: []

------------ Columna: banios/2 --------------- 
  Unidad: ' medios baños' -> Ejemplos: ['3 medios baños' '4 medios baños' '2 medios baños']
  Unidad: ' medio baño' -> Ejemplos: ['1 medio baño']

------------ Columna: latitud --------------- 
  Unidad: '-,' -> Ejemplos: []
  Unidad: '-.' -> Ejemplos: ['-12.139912400000000' '-12.139271799999999' '-12.139554600000000']

------------ Columna: antiguedad --------------- 
  Unidad: 'Año de constru

Aunque bastaria con buscar unicamente los digitos con la expresion regular, esta vez vamos a considerar la estructura completa para cada variable para tener un mayor control de los valores que se van a convertir a enteros. 

In [81]:
def integer_format(exp: re.search,df: pd.DataFrame,columna: str) -> list:
    '''
    :param exp: Expresion regular que buscara un patron especifico segun la estructura de la variable. 
    :param df: Dataframe con los datos a procesar 
    :param columna: Variable a procesar 
    :return: Una lista con los valores convertidos a enteros segun la expresion regular. 
    '''
    valores = df[columna].values
    valores_f = []
    for valor in valores:
        if valor is None:
            valores_f.append(None)
            continue
        try:
            match = exp.findall(valor.lower())
            valor_f = match[0] if type(match[0]) is not tuple else reduce(lambda x, y: x+y, match[0])
            valores_f.append(int(valor_f))
        except IndexError:
            logger.warning(f"Formato incorrecto en la columna '{columna}' para el valor '{valor.lower()}'")
            valores_f.append(None)
        except ValueError:
            logger.warning(f" {valor} no se puede convertir a entero en '{columna}'")
            valores_f.append(None)

    return valores_f

Con la funcion anterior podemos convertir las variables a su respectivo formato...

In [87]:
def formateo(df: pd.DataFrame) -> pd.DataFrame:
    '''
    :param df: Dataframe con las variables a formatear
    :return: Dataframe con las variables formateadas
    '''
    df_copia = df.copy()
    df_copia[['latitud', 'longitud']] = coordenadas_format(df_copia, ['latitud', 'longitud'])
    df_copia['dormitorios'] = integer_format(re.compile(r"(\d+) dorm."), df_copia, "dormitorios")
    df_copia['banos'] = integer_format(re.compile(r"(\d+) baño"), df_copia, "banos")
    df_copia['banios/2'] = integer_format(re.compile(r"(\d+) medios? baños?"), df_copia, "banios/2")
    df_copia['antiguedad'] = integer_format(re.compile(r"(\d+) años|año de construcción\n(\d+)"), df_copia, "antiguedad")
    df_copia['area_total'] = integer_format(re.compile(r"(\d+) m²"), df_copia, "area_total")
    df_copia['area_cubierta'] = integer_format(re.compile(r"(\d+) m²"), df_copia, "area_cubierta")
    df_copia['estacionamiento'] = integer_format(re.compile(r"(\d+) estac."), df_copia, "estacionamiento")
    df_copia['precio_soles'] = integer_format(re.compile(r"s/\s*(\d+)|us\$(\d+)"), df_copia, "precio_soles")
    df_copia['precio_dolares'] = integer_format(re.compile(r"usd\s*(\d+)"), df_copia, "precio_dolares")
    df_copia['fecha_publicacion'] = integer_format(re.compile(r"(\d+) días?"), df_copia, "fecha_publicacion")
    df_copia[['latitud','longitud']] = df_copia[['latitud','longitud']].astype(float)
    
    return df_copia

In [88]:
data_formateada = formateo(data_preformato)

In [84]:
data_formateada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531 entries, 0 to 5530
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fecha_publicacion  5528 non-null   object 
 1   banios/2           890 non-null    float64
 2   latitud            3955 non-null   float64
 3   antiguedad         3839 non-null   float64
 4   dormitorios        5300 non-null   float64
 5   area_cubierta      3116 non-null   float64
 6   precio_dolares     2441 non-null   float64
 7   longitud           3955 non-null   float64
 8   banos              5251 non-null   float64
 9   descripcion        5450 non-null   object 
 10  estacionamiento    2104 non-null   float64
 11  area_total         5166 non-null   float64
 12  precio_soles       5424 non-null   float64
 13  distrito           5531 non-null   object 
 14  moneda             5424 non-null   object 
dtypes: float64(11), object(4)
memory usage: 648.3+ KB


In [91]:
def guardar_excel(df_format, ruta_local):
    df_format.to_csv(ruta_local, index=False)
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    ruta_folder_bucket = f"precios-depas-lima/csv/data-formateada_{timestamp_str}.csv"
    subir_archivo_a_space(ruta_local,ruta_folder_bucket)
    return ruta_folder_bucket

In [92]:
guardar_excel(data_formateada, "../csv/data_formateada.csv")

2025-02-17 22:30:11,949 - INFO - Archivo '../csv/data_formateada.csv' subido correctamente a 'bucket-proyectos' como 'precios-depas-lima/csv/data-formateada.csv'.


'precios-depas-lima/csv/data-formateada.csv'